## Import libraries

In [1]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

from pyomo.network import Arc, SequentialDecomposition

from idaes.core.util.initialization import propagate_state

# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

import idaes.logger as idaeslog
from pyomo.opt import TerminationCondition, SolverStatus

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Import unit models from the model library
from idaes.generic_models.unit_models import Flash

# Import R32, R125, IL info file
from HFCS_emimTF2N_PR import configuration

## Create Model

In [2]:
# Create the ConcreteModel and the FlowsheetBlock, and attach the flowsheet block to it.
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False}) 

# Add properties parameter blocks to the flowsheet with specifications
m.fs.props = GenericParameterBlock(default=configuration)

In [3]:
m.fs.F101 = Flash(default={"property_package": m.fs.props,
                               "has_heat_transfer": True,
                               "has_pressure_change": True})

# Call the degrees_of_freedom function, get intitial DOF
DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

The initial DOF is 8


In [4]:
## Update conditions to reduce DOF

# Fix Heater H101 inlet conditions
m.fs.F101.inlet.mole_frac_comp[0, "R32"].fix(0.5)
m.fs.F101.inlet.mole_frac_comp[0, "R125"].fix(0.5)
m.fs.F101.inlet.mole_frac_comp[0, "emimTf2N"].fix(0.001)

m.fs.F101.inlet.flow_mol[0].fix(1)
m.fs.F101.inlet.temperature.fix(300)
m.fs.F101.inlet.pressure.fix(200000)

# Fix F101 Flash outlet conditions
m.fs.F101.vap_outlet.temperature.fix(300)
m.fs.F101.deltaP.fix(0)

DOF_final = degrees_of_freedom(m)
print("The final DOF is {0}".format(DOF_final))

The final DOF is 0


In [5]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

## Calculate flash data

In [6]:
# Set pressures to loop over
P = [200000,500000]

# Set fraction of feed compositions of HFCs to loop over
HFC_feed_frac = [0.001,0.4,0.72,0.91,0.99]

# Make outlet arrays
IL_x = np.zeros((len(P),len(HFC_feed_frac)))
IL_y = np.zeros((len(P),len(HFC_feed_frac)))
R32_x = np.zeros((len(P),len(HFC_feed_frac)))
R32_y = np.zeros((len(P),len(HFC_feed_frac)))
R125_x = np.zeros((len(P),len(HFC_feed_frac)))
R125_y = np.zeros((len(P),len(HFC_feed_frac))) 

for i in range(len(P)):
    
    # Fix pressure
    m.fs.F101.inlet.pressure.fix(P[i])
    
    for j in range(len(HFC_feed_frac)):
        
        # Fix inlet mole fractions
        fract = 1 - HFC_feed_frac[j] 
        m.fs.F101.inlet.mole_frac_comp[0, "R32"].fix(0.3*HFC_feed_frac[j])
        m.fs.F101.inlet.mole_frac_comp[0, "R125"].fix(0.3*fract)
        m.fs.F101.inlet.mole_frac_comp[0, "emimTf2N"].fix(0.7)   

        m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)              

        # solve the model
        status = solver.solve(m, tee = False)
        # model.display()
        
        # If solution is optimal store the concentration, and calculated temperatures in the created arrays
        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):

            IL_x[i,j] = value(m.fs.F101.liq_outlet.mole_frac_comp[0, "emimTf2N"])
            IL_y[i,j] = value(m.fs.F101.vap_outlet.mole_frac_comp[0, "emimTf2N"])
            R32_x[i,j] = value(m.fs.F101.liq_outlet.mole_frac_comp[0, "R32"]) 
            R32_y[i,j] = value(m.fs.F101.vap_outlet.mole_frac_comp[0, "R32"])
            R125_x[i,j] = value(m.fs.F101.liq_outlet.mole_frac_comp[0, "R125"])
            R125_y[i,j] = value(m.fs.F101.vap_outlet.mole_frac_comp[0, "R125"])

            #Change this to feasible
            print("Running pressure:",P[i],"and HFC feed fraction",HFC_feed_frac[j])
#             print(value(m.fs.F101.liq_outlet.flow_mol[0]))
#             print(value(m.fs.F101.vap_outlet.flow_mol[0]))
        else:
            print('Infeasible.')

    

Running pressure: 200000 and HFC feed fraction 0.001
Running pressure: 200000 and HFC feed fraction 0.4
Running pressure: 200000 and HFC feed fraction 0.72
Running pressure: 200000 and HFC feed fraction 0.91
Running pressure: 200000 and HFC feed fraction 0.99
Running pressure: 500000 and HFC feed fraction 0.001
Running pressure: 500000 and HFC feed fraction 0.4
Running pressure: 500000 and HFC feed fraction 0.72
Running pressure: 500000 and HFC feed fraction 0.91
Running pressure: 500000 and HFC feed fraction 0.99


In [7]:
print("IL_x =", IL_x)
print("IL_y =", IL_y)
print("R32_x =", R32_x)
print("R32_y =", R32_y)
print("R125_x =", R125_x)
print("R125_y =", R125_y)

IL_x = [[0.89586162 0.86955959 0.84575128 0.82981626 0.82252284]
 [0.74237231 0.70522703 0.70000003 0.70000001 0.70000001]]
IL_y = [[1.e-08 1.e-08 1.e-08 1.e-08 1.e-08]
 [1.e-08 1.e-08 1.e-08 1.e-08 1.e-08]]
R32_x = [[1.54781545e-04 6.45950874e-02 1.21664796e-01 1.59213254e-01
  1.76229263e-01]
 [2.80296892e-04 1.18836629e-01 2.15999985e-01 2.72999989e-01
  2.96999988e-01]]
R32_y = [[8.19003777e-04 3.48730440e-01 6.69063904e-01 8.86565077e-01
  9.86989842e-01]
 [6.25499792e-04 2.75797672e-01 5.87667328e-01 8.47619232e-01
  9.81930377e-01]]
R125_x = [[0.1039836  0.06584533 0.03258393 0.01097049 0.0012479 ]
 [0.25734739 0.17593634 0.08399999 0.027      0.003     ]]
R125_y = [[0.999181   0.65126956 0.3309361  0.11343492 0.01301016]
 [0.9993745  0.72420233 0.41233267 0.15238077 0.01806962]]


In [ ]:
# points_x = []

# for i in range(len(IL_z)):
#     points = (IL_x[i], R32_x[i], R125_x[i])
#     points_x.append(points)
    
# print(points_x)

# points_y = []

# for i in range(len(IL_z)):
#     points = (IL_y[i], R32_y[i], R125_y[i])
#     points_y.append(points)
    
# print(points_y)

In [ ]:
# points_x = []

# for i in range(0,4): #to shift how points are printed, shift the order
#     points = (IL_x[i]*100, R125_x[i]*100, R32_x[i]*100)
#     points_x.append(points)

# points_y = []

# for i in range(0,4): #to shift how points are printed, shift the order
#     points = (IL_y[i]*100, R125_y[i]*100, R32_y[i]*100)
#     points_y.append(points)

# import ternary

# scale = 100
# figure, tax = ternary.figure(scale=scale)

# # Draw Boundary and Gridlines
# tax.boundary(linewidth=2.0)
# tax.gridlines(color="blue", multiple=5)

# # Set Axis labels and Title
# fontsize = 12
# offset = 0.14
# # tax.set_title("Ternary phase diagram 8 bar\n", fontsize=24)
# tax.left_axis_label("R32", fontsize=fontsize, offset=offset)
# tax.right_axis_label("R125", fontsize=fontsize, offset=offset)
# tax.bottom_axis_label("IL", fontsize=fontsize, offset=offset)

# #for pressure changing plot would need to draw line between x[0] and x[final] for the indivual pressure
# #use tax.line with the tax.scatter below
# for i in range(4):
#     tax.line(points_x[i], points_y[i], linewidth=3., color='green', linestyle=":")

# # make this tax.line
# tax.scatter(points_x, marker='s', color='red', label="liquid")
# #Comment out for pressure changing plot
# tax.scatter(points_y, marker='s', color='blue', label="vapor")


# tax.ticks(axis='lbr', multiple=10, linewidth=1, offset=0.025)
# tax.get_axes().axis('off')
# tax.clear_matplotlib_ticks()
# tax.show()